# Import

In [1]:
from optimum.onnxruntime import ORTModelForFeatureExtraction, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from onnxruntime import InferenceSession
from onnxruntime.quantization import QuantType
from transformers import AutoFeatureExtractor
import torch

from PIL import Image
import time

# Convert Onnx

## Vit google

In [2]:
vit_gg_onnx = ORTModelForFeatureExtraction.from_pretrained('/app/nfs_clientshare/mew/project/Similarity_model/weights/vitgg_lr2e05_ep3_loss0.0', export=True)


Framework not specified. Using pt to export to ONNX.
/home/music/.local/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu121
/home/music/.local/lib/python3.8/site-packages/transformers/models/vit/modeling_vit.py:170: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:
/home/music/.local/lib/python3.8/site-packages/transformers/models/vit/modeling_vit.py:176: TracerWarning: Converting a tensor to a Python boolean might

In [2]:
# convert to onnx
vit_gg_onnx = ORTModelForFeatureExtraction.from_pretrained('/app/nfs_clientshare/mew/project/Similarity_model/weights/vitgg_lr2e05_ep3_loss0.0', export=True)
processor_vit_gg_onnx = AutoFeatureExtractor.from_pretrained('/app/nfs_clientshare/mew/project/Similarity_model/weights/vitgg_lr2e05_ep3_loss0.0')

Framework not specified. Using pt to export to ONNX.


In [4]:
# quantization
quantizer = ORTQuantizer.from_pretrained(vit_gg_onnx)
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
dqconfig.weights_dtype = QuantType.QUInt8

In [2]:
save_directory = "/home/music/Desktop/measure_model/models/"

In [6]:
# save model onnx
vit_gg_onnx.save_pretrained(save_directory+"vit_gg_onnx")

In [7]:
# save model onnx quantized
model_quantized_path = quantizer.quantize(
    save_dir=save_directory+"vit_gg_onnx_quantize",
    quantization_config=dqconfig,
)

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/u8, channel-wise: False)
Quantizing model...
Saving quantized model at: /home/music/Desktop/measure_model/models/vit_gg_onnx_quantize (external data format: False)
Configuration saved in /home/music/Desktop/measure_model/models/vit_gg_onnx_quantize/ort_config.json


# test onnx

In [3]:
img = Image.open("/home/music/Desktop/measure_model/data/image_net/n01514668_cock.JPEG")

## ViT google

In [4]:
# vit google onnx
vit_gg_onnx_path = save_directory+"vit_gg_onnx/model.onnx"
vit_gg_onnx = InferenceSession(vit_gg_onnx_path, providers=['CUDAExecutionProvider'])
processor_vit_gg_onxx = AutoFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

/home/music/.local/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/home/music/.local/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
inputs = processor_vit_gg_onxx(images=img.convert("RGB"), return_tensors="np")
st = time.time()
outputs_onnx = vit_gg_onnx.run(output_names=["last_hidden_state"], input_feed=dict(inputs))
delta = time.time() - st
print(f"runtime : {delta*1000} ms")

runtime : 325.0598907470703 ms


In [18]:
# vit google onnx quantized
vit_gg_onnx_quantized_path = "/home/music/Desktop/measure_model/models/vit_gg_onnx_quantize"
vit_gg_onnx_quantized_model_path = vit_gg_onnx_quantized_path+"/model_quantized.onnx"
vit_gg_onnx_quantized = InferenceSession(vit_gg_onnx_quantized_model_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
processor_vit_gg_onxx = AutoFeatureExtractor.from_pretrained(vit_gg_onnx_quantized_path)

In [19]:
inputs = processor_vit_gg_onxx(images=img.convert("RGB"), return_tensors="np")
st = time.time()
outputs_onnx_quantize = vit_gg_onnx_quantized.run(output_names=["last_hidden_state"], input_feed=dict(inputs))
delta = time.time() - st
print(f"runtime : {delta*1000} ms")

runtime : 307.5551986694336 ms


In [14]:
# Original ViT google
from transformers import ViTImageProcessor, ViTModel
vit_gg = ViTModel.from_pretrained('/app/nfs_clientshare/mew/project/Similarity_model/weights/vitgg_lr2e05_ep3_loss0.0')
processor_vit_gg = ViTImageProcessor.from_pretrained('/app/nfs_clientshare/mew/project/Similarity_model/weights/vitgg_lr2e05_ep3_loss0.0')
vit_gg.eval().to('cpu')
inputs = processor_vit_gg(images=img, return_tensors="pt").to('cpu')

In [15]:
start_time_torch = time.time()
outputs = vit_gg(**inputs)
delta_time_torch = time.time() - start_time_torch
print("runtime :", delta_time_torch*1000, "ms")

runtime : 472.4152088165283 ms
